In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

from fiona.crs import from_epsg
import shapely
from shapely.geometry import *


from geopy.distance import vincenty

#### Shape file

In [2]:
nybb = gpd.read_file('nybb_17c/nybb.shp')

In [3]:
# # >>> from geopy.distance import vincenty
# newport_ri = (41.49008, -71.312796)
# cleveland_oh = (41.499498, -81.695391)
# print(vincenty(newport_ri, cleveland_oh).kilometers)


In [4]:
# Get the route files
# import subprocess
# subprocess.run("powershell Invoke-WebRequest {} -OutFile {}".format('http://data.mytransit.nyc.s3.amazonaws.com/gtfs/2015/gtfs_nyct_bus_20150103.zip', 'gtfs_nyct_bus_20150103.zip'), shell=True)
# subprocess.run("powershell Invoke-WebRequest {} -OutFile {}".format('http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150128.csv.xz', 'bus_time_20150128.csv.xz'), shell=True)
# subprocess.run("powershell Invoke-WebRequest {} -OutFile {}".format('http://data.mytransit.nyc.s3.amazonaws.com/bus_time/2015/2015-01/bus_time_20150129.csv.xz', 'bus_time_20150129.csv.xz'), shell=True)


# import zipfile
# import lzma
# zip = zipfile.ZipFile('gtfs_nyct_bus_20150103.zip')
# zip.extractall()

In [5]:
def convert_to_int64(row):
    try:
        return np.int64(row)
    except ValueError:
        return np.nan

def utc_to_est(row):
    try:
        return row - timedelta(hours=5)
    except:
        raise

csv_f = pd.read_csv
archive = pd.concat([csv_f("bus_time_20150128.csv"),
                     csv_f("bus_time_20150129.csv")])        
        
trips = pd.read_csv("trips.txt")
stops = pd.read_csv("stops.txt")
schedules = pd.read_csv("stop_times.txt")

/home/sunglyoung/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
archive.next_stop_id = archive.next_stop_id.apply(convert_to_int64)

In [7]:
#query and clean
live_archive = archive[archive.block_assigned > 0] # "assigned" to a route
today = live_archive[live_archive.service_date == 20150128] # due to time shift we get some dates in 1/27 and 1/29
#throw away trips with <15 reports
good_trips_only = today.groupby(today.trip_id).filter(lambda group: len(group) > 15)

#write this as our "clean" dataset
good_trips_only.to_csv("realtime.csv")

#merged dataframe with both 
partial = good_trips_only.merge(trips, on='trip_id') 
df = partial.merge(stops, left_on="next_stop_id", right_on="stop_id")

In [101]:
# Select one bus route 
pd.options.display.max_columns = None
m15_only = df[df.route_id.str.contains("M15")].reset_index()
m15_only["timestamp"] = pd.to_datetime(m15_only["timestamp"])

In [102]:
m15_reg = m15_only[m15_only.trip_id.str.contains("M15")].reset_index()
m15_sbs = m15_only[m15_only.trip_id.str.contains("SBS")].reset_index()

In [103]:
print('The number of regular M15 buses over 48 hours =',len(m15_reg.trip_id.unique()))
print('The number of express M15 buses over 48 hours =',len(m15_sbs.trip_id.unique()))

('The number of regular M15 buses over 48 hours =', 257)
('The number of express M15 buses over 48 hours =', 335)


## Analysis

In [216]:
def split_direction(df):
    df_1 = df[df.direction_id == 1]
    
    df_1 = df_1[df_1.trip_id == df_1.trip_id.values[0]]
    
    df_1.sort_values('timestamp')
    
    df_0 = df[df.direction_id == 0]
    
    df_0 = df_0[df_0.trip_id == df_0.trip_id.values[0]]
      
    df_0 = df_0.sort_values('timestamp')
    return (df_1, df_0)


#### It will split DataFrame into two bus's direction

In [217]:
m15_reg_1, m15_reg_0 = split_direction(m15_reg)

In [218]:
m15_sbs_1, m15_sbs_0 = split_direction(m15_sbs)

#### The typical NYC block length is 0.05 miles ~ 100 meters

## We need 1 and 2 for each direction!!!!

In [226]:
# >>> from geopy.distance import vincenty
def finding_busstop(df1_regular, df2_express, meters): # df1 is regular, df2 is express, meters is threshold
    

    #### cleaning filtered data by stop_name             for sharing stop in meters
    
    clean_df1 = df1_regular.drop_duplicates(subset = 'stop_name', keep = 'first').reset_index(drop = True)
    clean_df2 = df2_express.drop_duplicates(subset = 'stop_name', keep = 'first').reset_index(drop = True)
    
    #### finding shared busstops
    regular = []
    express = []

    
    for j in range(len(clean_df1)):
        sStop1 = (clean_df1.latitude[j], clean_df1.longitude[j])
    
        for i in range(len(clean_df2)):
            rStop1 = (clean_df2.latitude[i], clean_df2.longitude[i])
            dist = (vincenty(rStop1, sStop1).meters)
        
            if(dist <= meters):
            
                regular.append(j)
                express.append(i)
    ##### making a bus stop panda frame
    cols = {'regular_stop' : regular, 'express_stop' : express}
    EStops = pd.DataFrame(cols)
    EStops.drop_duplicates(subset = 'express_stop', keep = 'first', inplace= True)
    regular = EStops.regular_stop
    express = EStops.express_stop
    
    ###### Finding the stop name  
    Share_Stop = clean_df1.iloc[regular,20].reset_index(drop = True)
    Share_eStop = clean_df2.iloc[express,20].reset_index(drop = True)
        
    
    return (Share_Stop, Share_eStop)




#### Finding Share_Stop and Share_eStop for analysis

In [261]:
Share_Stop, Share_eStop  = finding_busstop(m15_reg_1, m15_sbs_1, 100)

In [262]:
Share_Stop

0             2 AV/E 125 ST
1             2 AV/E 116 ST
2              2 AV/E 96 ST
3              2 AV/E 66 ST
4              2 AV/E 46 ST
5              2 AV/E 39 ST
6              2 AV/E 31 ST
7         ALLEN ST/CANAL ST
8        WATER ST/MAIDEN LA
9    WATER ST/GOUVERNEUR LA
Name: stop_name, dtype: object

In [263]:
Share_eStop

0              2 AV/E 125 ST
1              2 AV/E 106 ST
2              2 AV/E 100 ST
3               2 AV/E 57 ST
4               2 AV/E 42 ST
5               2 AV/E 34 ST
6               2 AV/E 28 ST
7    MADISON ST/CATHERINE ST
8           WATER ST/WALL ST
9       SOUTH FERRY/TERMINAL
Name: stop_name, dtype: object

In [248]:
def r_anaysis(df, Share_Stop):
### cleaning the clean_df1 and clean_df2 for only sharing bus_stop
    clean_df1 = df.drop_duplicates(subset = 'stop_name', keep = 'first').reset_index(drop = True)
    SR = clean_df1[clean_df1.stop_name.isin(Share_Stop)].reset_index(drop = True) ## Sharing regular bus stop
#     SE = clean_df2[clean_df2.stop_name.isin(Share_eStop)].reset_index(drop = True) ## sharing expresss bus stop
#     df_join = SR.join(SE, lsuffix = '_regular', rsuffix = '_express')
    SR.dropna(axis = 0)
    
    
    #### calculating speed
#     express_speed = []
    regular_speed = []

    for i in range(len(SR)-1):
##             for SBS bus
#             ed = df_join.dist_along_route_express.values.astype(float)[i+1] \
#             - df_join.dist_along_route_express.values.astype(float)[i]
        
#             et = (df_join.timestamp_express[i+1] - df_join.timestamp_express[i]).seconds
#             express_speed.append(((ed/1000 * 0.625) /et) * 3600)
            
            # Regular bus

            rd = SR.dist_along_route.values.astype(float)[i+1] \
            - SR.dist_along_route.values.astype(float)[i]
        
            rt = (SR.timestamp[i+1] - SR.timestamp[i]).seconds
            regular_speed.append(((rd/1000 * 0.625) / rt) * 3600)
    
    ### making average_speed data frame
 
    cols = {'regular_mph' : regular_speed}
    mph = pd.DataFrame(cols)
    df_regular =  SR.join(mph)
    return (df_regular)
    
    

### regular _ Analysis

In [250]:
r_anaysis(m15_reg, Share_Stop )

,level_0,index,timestamp,vehicle_id,latitude,longitude,bearing,progress,service_date,trip_id,block_assigned,next_stop_id,dist_along_route,dist_from_stop,route_id,service_id,trip_headsign,direction_id,shape_id,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,regular_mph
0,13525,175834,2015-01-28 06:04:36,6786,40.707985,-74.003501,224.04,0,20150128,OH_A5-Weekday-SDon-002000_M15_2,1,403590,12826.96,56.99,M15,OH_A5-Weekday-SDon,SOUTH FERRY via 2 AV,1,M150351,403590,PEARL ST/FULTON ST,NaN,40.707630,-74.003998,NaN,NaN,0,NaN,-130.432500
1,16277,458195,2015-01-28 06:07:55,6702,40.708046,-74.003438,44.04,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,401669,1290.93,148.21,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,401669,PEARL ST/PECK SLIP,NaN,40.708969,-74.002174,NaN,NaN,0,NaN,11.850279
2,18701,460667,2015-01-28 06:13:44,6702,40.717909,-73.990921,62.72,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,401679,3129.04,200.18,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,401679,ALLEN ST/DELANCEY ST,NaN,40.719559,-73.989967,NaN,NaN,0,NaN,12.880080
3,19385,461351,2015-01-28 06:15:49,6702,40.724666,-73.987442,54.01,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,405098,3844.6,104.2,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,405098,1 AV / 4 ST,NaN,40.725437,-73.986725,NaN,NaN,0,NaN,6.708582
4,22021,463987,2015-01-28 06:25:52,6702,40.739401,-73.976705,54.02,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,405396,5642.5,30.52,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,405396,1 AV/ E 27 ST,NaN,40.739597,-73.976425,NaN,NaN,0,NaN,25.457344
5,22184,464273,2015-01-28 06:26:56,6702,40.744586,-73.972927,53.88,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,401696,6366.62,96.11,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,401696,1 AV/E 36 ST,NaN,40.745293,-73.972260,NaN,NaN,0,NaN,24.719879
6,22470,464559,2015-01-28 06:27:58,6702,40.749520,-73.969265,53.71,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,401699,7047.79,146.19,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,401699,1 AV/E 45 ST,NaN,40.750664,-73.968407,NaN,NaN,0,NaN,14.351155
7,23558,465647,2015-01-28 06:30:36,6702,40.757436,-73.963557,53.70,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,401704,8055.56,149.45,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,401704,1 AV/E 57 ST,NaN,40.758564,-73.962585,NaN,NaN,0,NaN,27.939919
8,24204,466433,2015-01-28 06:31:07,6702,40.758660,-73.962668,53.83,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,404253,8440.51,379.03,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,404253,1 AV/E 62 ST,NaN,40.761600,-73.960388,NaN,NaN,0,NaN,11.506181
9,25631,467860,2015-01-28 06:33:14,6702,40.765021,-73.958027,54.24,0,20150128,OH_A5-Weekday-SDon-006000_M15_1,1,401708,9089.97,220.41,M15,OH_A5-Weekday-SDon,EAST HARLEM 125 ST via 1 AV,0,M150350,401708,1 AV/E 70 ST,NaN,40.766705,-73.956642,NaN,NaN,0,NaN,17.763395


In [251]:
def e_anaysis(df, Share_eStop):
### cleaning the clean_df1 and clean_df2 for only sharing bus_stop
#     SR = clean_df1[clean_df1.stop_name.isin(Share_Stop)].reset_index(drop = True) ## Sharing regular bus stop
    clean_df2 = df.drop_duplicates(subset = 'stop_name', keep = 'first').reset_index(drop = True)
    SE = clean_df2[clean_df2.stop_name.isin(Share_eStop)].reset_index(drop = True) ## sharing expresss bus stop
#     df_join = SR.join(SE, lsuffix = '_regular', rsuffix = '_express')
#     df_join.dropna(axis = 0)
    
    
    #### calculating speed
    express_speed = []
#     regular_speed = []

    for i in range(len(SE)-1):
            # for SBS bus
            ed = SE.dist_along_route.values.astype(float)[i+1] \
            - SE.dist_along_route.values.astype(float)[i]
#         
            et = (SE.timestamp[i+1] - SE.timestamp[i]).seconds
            express_speed.append(((ed/1000 * 0.625) /et) * 3600)
            
#             # Regular bus

#             rd = df_join.dist_along_route_regular.values.astype(float)[i+1] \
#             - df_join.dist_along_route_regular.values.astype(float)[i]
        
#             rt = (df_join.timestamp_regular[i+1] - df_join.timestamp_regular[i]).seconds
#             regular_speed.append(((rd/1000 * 0.625) / rt) * 3600)
    
    ### making average_speed data frame
 
    cols = {'express_mph' : express_speed}
    mph = pd.DataFrame(cols)
    df_express = SE.join(mph)
    
    return(df_express)
    

In [268]:
e_anaysis(m15_sbs_1, Share_eStop)

,level_0,index,timestamp,vehicle_id,latitude,longitude,bearing,progress,service_date,trip_id,block_assigned,next_stop_id,dist_along_route,dist_from_stop,route_id,service_id,trip_headsign,direction_id,shape_id,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,express_mph
0,46,161895,2015-01-28 09:46:18,1273,40.803411,-73.933053,157.83,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,401738,180.23,126.16,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,401738,2 AV/E 125 ST,NaN,40.802494,-73.933891,NaN,NaN,0,NaN,15.252405
1,5775,167624,2015-01-28 09:59:53,1273,40.765929,-73.960459,233.73,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,401765,5704.99,883.88,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,401765,2 AV/E 57 ST,NaN,40.759022,-73.965652,NaN,NaN,0,NaN,12.336341
2,8471,170320,2015-01-28 10:03:33,1273,40.753940,-73.969202,233.99,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,401771,6911.21,566.84,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,401771,2 AV/E 42 ST,NaN,40.749527,-73.972572,NaN,NaN,0,NaN,13.138977
3,12622,174654,2015-01-28 10:18:13,1273,40.715682,-73.992322,242.87,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,402112,12050.01,770.03,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,402112,MADISON ST/CATHERINE ST,NaN,40.712097,-73.997284,NaN,NaN,0,NaN,-0.118248
4,32731,791726,2015-01-28 10:05:38,1273,40.749435,-73.972489,233.65,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,401774,7548.97,632.22,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,401774,2 AV/E 34 ST,NaN,40.744507,-73.976234,NaN,NaN,0,NaN,-0.155210
5,41974,1831899,2015-01-28 09:48:23,1273,40.796655,-73.938026,234.01,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,405347,1660.3,743.33,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,405347,2 AV/E 106 ST,NaN,40.790859,-73.942429,NaN,NaN,0,NaN,5.933452
6,42401,1832326,2015-01-28 09:51:32,1273,40.787643,-73.944623,233.82,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,903103,2158.71,96.08,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,903103,2 AV/E 100 ST,NaN,40.786915,-73.945244,NaN,NaN,0,NaN,13.515648
7,42884,1832809,2015-01-28 10:07:44,1273,40.744327,-73.976212,233.91,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,403525,7997.47,431.89,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,403525,2 AV/E 28 ST,NaN,40.740971,-73.978760,NaN,NaN,0,NaN,12.193013
8,43633,1833558,2015-01-28 10:23:58,1273,40.706453,-74.005577,214.75,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,403754,13275.69,241.48,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,403754,WATER ST/WALL ST,NaN,40.705044,-74.007759,NaN,NaN,0,NaN,22.641786
9,43887,1833832,2015-01-28 10:25:01,1273,40.704906,-74.007790,214.90,0,20150128,OF_A5-Weekday-SDon-028500_SBS15_1,1,803120,13909.66,621.7,M15+,OF_A5-Weekday-SDon,SELECT BUS SERVICE SOUTH FERRY via 2 AV,1,SBS150060,803120,SOUTH FERRY/TERMINAL,NaN,40.702122,-74.013664,NaN,NaN,0,NaN,NaN


In [276]:
def final_df(df_regular, df_express, Share_Stop = Share_Stop, Share_eStop = Share_eStop):

    my_a_0 = []

    for group in df_regular.groupby('trip_id'):
    
        a = r_anaysis(group[1], Share_Stop)
        my_a_0.append(a)
    my_df_r = pd.concat((my_a_0), axis=0)
    
    my_a_1 = []

    for group in df_express.groupby('trip_id'):
    
        a = e_anaysis(group[1], Share_eStop)
        my_a_1.append(a)
    my_df_e = pd.concat((my_a_1), axis=0)
    
    return (my_df_r, my_df_e)

In [277]:
my_df_r, my_df_e = final_df(m15_reg, m15_sbs)

In [279]:
my_df_r.shape

(1006, 29)

In [280]:
my_df_e.shape

(1734, 29)